## Energy Label Classification

### Consolidating the collected information - Part 2

We saw previously that we are mainly interested in the __features__ column from the CSV we created out of the HTT requests. 

Look into the respective column more closely and make sure we're discarding as much junk as possible.

In [1]:
from datetime import datetime
import time

import multiprocessing as mp
import os
import ast
import json

import utils

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)
import pandas_explode 
pandas_explode.patch() # adds a `df.explode` method to all DataFrames 

In [2]:
energy_cls_file = os.path.join(os.getcwd(), 'data', 'energy_classes.csv')
energy_cls = pd.read_csv(energy_cls_file)

energy_cls.columns = ['initial_index', 'features']
energy_cls.tail(1)

,initial_index,features
38213,616,[]


In [3]:
energy_cls.dtypes

initial_index     int64
features         object
dtype: object

In [6]:
energy_cls = energy_cls[energy_cls['features']!='[]']
energy_cls.shape

(15034, 2)

In [14]:
energy_cls['features_len'] = energy_cls['features'].apply(lambda x: len(str(x)))
energy_cls = energy_cls[energy_cls['features_len']>4]
energy_cls.shape

(14992, 3)

In [15]:
energy_cls['features'] = energy_cls['features'].apply(lambda x: ast.literal_eval(x))

In [16]:
start = datetime.now()
energy_cls.drop(columns=['features_len'], inplace=True)
energy_cls_new = energy_cls.explode('features')
print(datetime.now()-start)

0:00:18.845139


In [19]:
energy_cls_new.head()

,initial_index,features
29,30,"{'type': 'Feature', 'id': '311107437', 'geometry': {'type': 'Point', 'coordinates': [12.485715793863, 55.628707247245]}, 'properties': {'EnergyLabelClassification': {'value': 'C'}, 'StreetName': {'value': 'Lille Friheden'}, 'HouseNumber': {'value': '1'}, 'ZipCode': {'value': '2650'}, 'CityName': {'value': 'Hvidovre'}, 'BBRUseCode': {'label': 'Type', 'value': 'Bygning til hotel, restaurant, vaskeri, frisør og anden servicevirksomhed'}, 'YearOfConstruction': {'label': 'Opført', 'value': '2003'..."
29,30,"{'type': 'Feature', 'id': '311248286', 'geometry': {'type': 'Point', 'coordinates': [12.484573114953, 55.627651771051]}, 'properties': {'EnergyLabelClassification': {'value': 'E'}, 'StreetName': {'value': 'Strandmarksvej'}, 'HouseNumber': {'value': '20'}, 'ZipCode': {'value': '2650'}, 'CityName': {'value': 'Hvidovre'}, 'BBRUseCode': {'label': 'Type', 'value': 'Bygning til handel, lager, kontor m.v.'}, 'YearOfConstruction': {'label': 'Opført', 'value': '1968'}, 'HeatSupply': {'label': 'Varme'..."
29,30,"{'type': 'Feature', 'id': '311365116', 'geometry': {'type': 'Point', 'coordinates': [12.485618025402, 55.62814356202]}, 'properties': {'EnergyLabelClassification': {'value': 'A'}, 'StreetName': {'value': 'Strandmarksvej'}, 'HouseNumber': {'value': '40'}, 'ZipCode': {'value': '2650'}, 'CityName': {'value': 'Hvidovre'}, 'BBRUseCode': {'label': 'Type', 'value': 'Etageejendom'}, 'YearOfConstruction': {'label': 'Opført', 'value': '1961, 1961, 1961, 1961, 1962, 1962, 1962, 1962, 1962, 1962, 1962, ..."
30,31,"{'type': 'Feature', 'id': '311107437', 'geometry': {'type': 'Point', 'coordinates': [12.485715793863, 55.628707247245]}, 'properties': {'EnergyLabelClassification': {'value': 'C'}, 'StreetName': {'value': 'Lille Friheden'}, 'HouseNumber': {'value': '1'}, 'ZipCode': {'value': '2650'}, 'CityName': {'value': 'Hvidovre'}, 'BBRUseCode': {'label': 'Type', 'value': 'Bygning til hotel, restaurant, vaskeri, frisør og anden servicevirksomhed'}, 'YearOfConstruction': {'label': 'Opført', 'value': '2003'..."
30,31,"{'type': 'Feature', 'id': '311365116', 'geometry': {'type': 'Point', 'coordinates': [12.485618025402, 55.62814356202]}, 'properties': {'EnergyLabelClassification': {'value': 'A'}, 'StreetName': {'value': 'Strandmarksvej'}, 'HouseNumber': {'value': '40'}, 'ZipCode': {'value': '2650'}, 'CityName': {'value': 'Hvidovre'}, 'BBRUseCode': {'label': 'Type', 'value': 'Etageejendom'}, 'YearOfConstruction': {'label': 'Opført', 'value': '1961, 1961, 1961, 1961, 1962, 1962, 1962, 1962, 1962, 1962, 1962, ..."


Extract the __geometry__ piece of information from the __features__ column.

In [28]:
energy_cls_new['geometry'] = energy_cls_new['features'].apply(lambda x: x['geometry'])
energy_cls_new.head(1)

,initial_index,features,geometry
29,30,"{'type': 'Feature', 'id': '311107437', 'geometry': {'type': 'Point', 'coordinates': [12.485715793863, 55.628707247245]}, 'properties': {'EnergyLabelClassification': {'value': 'C'}, 'StreetName': {'value': 'Lille Friheden'}, 'HouseNumber': {'value': '1'}, 'ZipCode': {'value': '2650'}, 'CityName': {'value': 'Hvidovre'}, 'BBRUseCode': {'label': 'Type', 'value': 'Bygning til hotel, restaurant, vaskeri, frisør og anden servicevirksomhed'}, 'YearOfConstruction': {'label': 'Opført', 'value': '2003'...","{'type': 'Point', 'coordinates': [12.485715793863, 55.628707247245]}"


#### Save the results in a file for further processing

In [30]:
energy_cls_file_clean = os.path.join(os.getcwd(), 'data', 'energy_classes_clean.csv')
energy_cls_new.to_csv(energy_cls_file_clean, sep=';', index=False)

Double check the results.

In [31]:
os.path.join(os.getcwd(), 'data', 'energy_classes_clean.csv')

'/home/osboxes/courses/ibm/data/energy_classes_clean.csv'

In [32]:
!head -n 2 '/home/osboxes/courses/ibm/data/energy_classes_clean.csv'

initial_index;features;geometry
30;{'type': 'Feature', 'id': '311107437', 'geometry': {'type': 'Point', 'coordinates': [12.485715793863, 55.628707247245]}, 'properties': {'EnergyLabelClassification': {'value': 'C'}, 'StreetName': {'value': 'Lille Friheden'}, 'HouseNumber': {'value': '1'}, 'ZipCode': {'value': '2650'}, 'CityName': {'value': 'Hvidovre'}, 'BBRUseCode': {'label': 'Type', 'value': 'Bygning til hotel, restaurant, vaskeri, frisør og anden servicevirksomhed'}, 'YearOfConstruction': {'label': 'Opført', 'value': '2003'}, 'HeatSupply': {'label': 'Varme', 'value': 'Fjernvarme'}, 'DEMOLink': {'value': 'http://energisparebygning.dk/DEMOService.svc/PdfDocument/311107437'}}};{'type': 'Point', 'coordinates': [12.485715793863, 55.628707247245]}
